In [1]:
import pandas as pd
import numpy as np
import util

enrollment_path = './data/tn/enrollment/'

years = [
    2013
    , 2014
    , 2015
    , 2016
    , 2017
    , 2018
    , 2019
]

enrollment_files = [
    'tn_enrollment_2013_cleaned.csv'
    , 'tn_enrollment_2014_cleaned.csv'
    , 'tn_enrollment_2015_cleaned.csv'
    , 'tn_enrollment_2016_cleaned.csv'
    , 'tn_enrollment_2017_cleaned.csv'
    , 'tn_enrollment_2018_cleaned.csv'
    , 'tn_enrollment_2019_cleaned.csv'
]

df = pd.read_csv(enrollment_path + enrollment_files[0])
df['year'] = df.apply(lambda x: years[0], axis=1)

for i in range(1, len(enrollment_files)):
    df2 = pd.read_csv(enrollment_path + enrollment_files[i])
    df2['year'] = df2.apply(lambda x: years[i], axis=1)
    df = df.append(df2, ignore_index = True, sort = True)

In [2]:
# rename All Race/Ethnic Groups to All Groups
df['temp'] = [x if x != 'All Race/Ethnic Groups' else 'All Groups' for x in df['group_state']]
del df['group_state']
df = df.rename(columns={"temp": "group_state"})

# drop rows with combined genders
df = df[df['gender'] != 'M & F']
df = df[df['gender'] != 'All Genders']

In [3]:
df.columns
# roll up values to get rid of gender detail
df = df.groupby(['year', 'district_id', 'district', 'school_id', 'school', 'grade', 'group_state'], as_index=False)['num'].sum()

In [4]:
# import 2012 values (All Grades only)
df_2012 = pd.read_csv(enrollment_path + 'tn_enrollment_2012_nogrades_cleaned.csv')

In [5]:
# transpose 2012 data
df_2012 = pd.melt(df_2012, id_vars=['year', 'district_id', 'district', 'school_id', 'school']
        , value_vars=['All Groups'
            , 'White'
            , 'African American'
            , 'Hispanic'
            , 'Asian'
            , 'Native American'
            , 'Free eligible'
            , 'Reduced eligible'
            , 'Free reduced eligible'
            ],
        var_name='group_state'
        , value_name='num')

# add grade column
df_2012['grade'] = df_2012.apply(lambda x: 'All Grades', axis=1)

In [6]:
 df = df.append(df_2012, ignore_index = True, sort=True)

In [9]:
def toInt(row, column):
    try:
        return int(row[column])
    except:
        return np.nan
df['num'] = df.apply(lambda x: toInt(x, 'num'), axis = 1)
print(df.shape)
df = df.dropna(subset=['num'])
df = df[df['num'] != 0]
print(df.shape)

(471661, 8)
(406535, 8)


In [10]:
print(df.dtypes)
df['district_id'] = df['district_id'].astype(str)
df['year'] = df['year'].astype(str)
df['school_id'] = df['school_id'].astype(str)
df['num'] = df['num'].astype('int64')
print(df.dtypes)

district        object
district_id      int64
grade           object
group_state     object
num            float64
school          object
school_id        int64
year             int64
dtype: object
district       object
district_id    object
grade          object
group_state    object
num             int64
school         object
school_id      object
year           object
dtype: object


In [12]:
# export results
df.to_csv('./data/finalized/tn_enrollment.csv', index=False)